In [1]:
!pip freeze | grep scikit-learn

scikit-learn==1.0.2


In [3]:
import pickle
import pandas as pd

**Script input**

In [16]:
year = 2021
month = 2

---

In [10]:
with open('model.bin', 'rb') as f_in:
    dv, lr = pickle.load(f_in)

In [11]:

categorical = ['PUlocationID', 'DOlocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.dropOff_datetime - df.pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [18]:

#df = read_data('https://nyc-tlc.s3.amazonaws.com/trip+data/fhv_tripdata_????-??.parquet')
df = read_data(f'../data/fhv_tripdata_{year:04d}-{month:02d}.parquet')

df.head()

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number,duration
1,B00021,2021-02-01 00:55:40,2021-02-01 01:06:20,173,82,None,B00021,10.666667
2,B00021,2021-02-01 00:14:03,2021-02-01 00:28:37,173,56,None,B00021,14.566667
3,B00021,2021-02-01 00:27:48,2021-02-01 00:35:45,82,129,None,B00021,7.950000
4,B00037,2021-02-01 00:12:50,2021-02-01 00:26:38,-1,225,None,B00037,13.800000
5,B00037,2021-02-01 00:00:37,2021-02-01 00:09:35,-1,61,None,B00037,8.966667


In [20]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = lr.predict(X_val)

**Q1. mean predicted duration**

In [21]:
y_pred.mean()

16.191691679979066

In [23]:
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

df.head()

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number,duration,ride_id
1,B00021,2021-02-01 00:55:40,2021-02-01 01:06:20,173,82,None,B00021,10.666667,2021/02_1
2,B00021,2021-02-01 00:14:03,2021-02-01 00:28:37,173,56,None,B00021,14.566667,2021/02_2
3,B00021,2021-02-01 00:27:48,2021-02-01 00:35:45,82,129,None,B00021,7.950000,2021/02_3
4,B00037,2021-02-01 00:12:50,2021-02-01 00:26:38,-1,225,None,B00037,13.800000,2021/02_4
5,B00037,2021-02-01 00:00:37,2021-02-01 00:09:35,-1,61,None,B00037,8.966667,2021/02_5


In [24]:
df_result = pd.DataFrame()
df_result['ride_id'] = df['ride_id']
df_result['duration_preds'] = y_pred

df_result.head()

,ride_id,duration_preds
1,2021/02_1,14.539865
2,2021/02_2,13.740422
3,2021/02_3,15.593339
4,2021/02_4,15.188118
5,2021/02_5,13.817206


In [26]:
output_file = f'../data/{year:04d}_{month:02d}_preds.parquet'

df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)

In [7]:
! du -ah ../data

 19M	../data/2021_02_preds.parquet
 10M	../data/fhv_tripdata_2021-02.parquet
 13M	../data/fhv_tripdata_2021-03.parquet
 42M	../data


Размер файла - 19М